In [1]:
#필수 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

In [2]:
# 인국공 실시간 주차정보 확인

import urllib.request
import xml.dom.minidom

encodingKey = "5J%2BydY16PVafkW5TJb6vB%2Fr1WS7SPflvRLPiw2JgJidn5%2B2uKOXNORGMiFkPyEGMgreC4%2BAXFkoa7z5o770eog%3D%3D"
decodingKey = "+ZZZ1B7mCisllsZL2HbT4yEsm72HdRmlNf2IcfIqxgUr228vYT/7dkwEJd+6Ek4eEIkGlex3lxerGNOYFE9Ndw=="

print("[START]")

# request url정의
url = "http://openapi.airport.kr/openapi/service/StatusOfParking/getTrackingParking?ServiceKey=" + encodingKey
request = urllib.request.Request(url)

# request보내기
response = urllib.request.urlopen(request)

# 결과 코드 정의
rescode = response.getcode()

if(rescode==200):
    response_body = response.read()
    dom = xml.dom.minidom.parseString(response_body.decode('utf-8'))
    pretty_xml_as_string = dom.toprettyxml()
    print(pretty_xml_as_string)
else:
    print("Error Code:" + rescode)

print("[END]")

[START]
<?xml version="1.0" ?>
<response>
	<header>
		<resultCode>00</resultCode>
		<resultMsg>NORMAL SERVICE.</resultMsg>
	</header>
	<body>
		<items>
			<item>
				<datetm>20210817105917.533</datetm>
				<floor>T1 단기주차장지상층</floor>
				<parking>65</parking>
				<parkingarea>993</parkingarea>
			</item>
			<item>
				<datetm>20210817105917.533</datetm>
				<floor>T1 단기주차장지하1층</floor>
				<parking>75</parking>
				<parkingarea>973</parkingarea>
			</item>
			<item>
				<datetm>20210817105917.533</datetm>
				<floor>T1 단기주차장지하2층</floor>
				<parking>64</parking>
				<parkingarea>1339</parkingarea>
			</item>
			<item>
				<datetm>20210817105917.531</datetm>
				<floor>T2 단기주차장지상1층</floor>
				<parking>327</parking>
				<parkingarea>988</parkingarea>
			</item>
			<item>
				<datetm>20210817105917.531</datetm>
				<floor>T2 단기주차장지상2층</floor>
				<parking>14</parking>
				<parkingarea>470</parkingarea>
			</item>
			<item>
				<datetm>20210817105917.531</datetm>
				<floor>T2 단기주차장지상3층</fl

In [3]:
#xml을 df로 변환
! pip install xmltodict 
import xmltodict

xmlObject = xmltodict.parse(pretty_xml_as_string)
dict_data = xmlObject['response']['body']['items']['item']
df1 = pd.DataFrame(dict_data)
df1.columns = ["datetm", "floor", "parking", "parkingarea"]

In [4]:
#데이터 확인
df1

,datetm,floor,parking,parkingarea
0,20210817105917.533,T1 단기주차장지상층,65,993
1,20210817105917.533,T1 단기주차장지하1층,75,973
2,20210817105917.533,T1 단기주차장지하2층,64,1339
3,20210817105917.531,T2 단기주차장지상1층,327,988
4,20210817105917.531,T2 단기주차장지상2층,14,470
5,20210817105917.531,T2 단기주차장지상3층,15,448
6,20210817105917.531,T2 단기주차장지상4층,0,486
7,20210817105917.531,T2 단기주차장지하M층,89,751
8,20210817105917.534,T1 장기 P1 주차장,258,2762
9,20210817105917.536,T1 장기 P1 주차타워,545,1374


In [5]:
#1시간 단위로 주차장 데이터 갱신 / 오전 11시 ~ 18시까지
import threading
import time
import xmltodict

parking_data = pd.DataFrame()

def train_check():
    import urllib.request
    import xml.dom.minidom

    encodingKey = "5J%2BydY16PVafkW5TJb6vB%2Fr1WS7SPflvRLPiw2JgJidn5%2B2uKOXNORGMiFkPyEGMgreC4%2BAXFkoa7z5o770eog%3D%3D"
    decodingKey = "+ZZZ1B7mCisllsZL2HbT4yEsm72HdRmlNf2IcfIqxgUr228vYT/7dkwEJd+6Ek4eEIkGlex3lxerGNOYFE9Ndw=="

    # request url정의
    url = "http://openapi.airport.kr/openapi/service/StatusOfParking/getTrackingParking?ServiceKey=" + encodingKey
    request = urllib.request.Request(url)

    # request보내기
    response = urllib.request.urlopen(request)

    # 결과 코드 정의
    rescode = response.getcode()

    if(rescode==200):
        response_body = response.read()
        dom = xml.dom.minidom.parseString(response_body.decode('utf-8'))
        pretty_xml_as_string = dom.toprettyxml()
    else:
        print("Error Code:" + rescode)
        
    xmlObject = xmltodict.parse(pretty_xml_as_string)
    dict_data = xmlObject['response']['body']['items']['item']
    df1 = pd.DataFrame(dict_data)
    df1.columns = ["datetm", "floor", "parking", "parkingarea"]
    
    #갱신된 데이터를 하나의 데이터프레임에 병합
    global parking_data
    parking_data = pd.concat([parking_data,df1])
    print(df1)

    threading.Timer(3600, train_check).start() #3600초 = 1시간

train_check()

               datetm          floor parking parkingarea
0  20210817105917.533    T1 단기주차장지상층      65         993
1  20210817105917.533   T1 단기주차장지하1층      75         973
2  20210817105917.533   T1 단기주차장지하2층      64        1339
3  20210817105917.531   T2 단기주차장지상1층     327         988
4  20210817105917.531   T2 단기주차장지상2층      14         470
5  20210817105917.531   T2 단기주차장지상3층      15         448
6  20210817105917.531   T2 단기주차장지상4층       0         486
7  20210817105917.531   T2 단기주차장지하M층      89         751
8  20210817105917.534   T1 장기 P1 주차장     258        2762
9  20210817105917.536  T1 장기 P1 주차타워     545        1374


In [6]:
#병합한 데이터를 csv로 저장
#8월 16일 오전 11시 ~ 18시 주차 현황 데이터
parking_data.to_csv('parking_data_time_hour.csv')